# WSI registration

methods to transform slide images of different stains that their coord systems overlay (?)

In [ ]:
import os

dir_path: str = "/home/sean/Desktop/MastherThesisDaten/DigiStrucMed_Braesen/Scans für QupathProjekt RCC (3 Fälle) 10062021/"
images: list[str] = []

for file in os.listdir(dir_path):
   filename = os.fsdecode(file)
   if file.endswith(".svs"):
      # print(os.fsencode(dir_path+filename))
      images.append(os.fsencode(dir_path+filename))

In [ ]:
from cucim import CuImage
from skimage.color import rgb2gray

import numpy as np
test_pics = []

for image in images[0:3]:
    pics.append(
        np.asarray(
            rgb2gray(
            CuImage(image).read_region(level=3)
            )))

## cucim - Daisy

In [ ]:
# from cucim import CuImage
# import pprint
# import numpy as np
# from cucim.skimage.feature import match_descriptors, plot_matches, daisy

# import matplotlib.pyplot as plt


In [ ]:
# img1, img2, img3 = test_pics
# descriptor_extractor = SIFT()

# descriptor_extractor.detect_and_extract(img1)
# keypoints1 = descriptor_extractor.keypoints
# descriptors1 = descriptor_extractor.descriptors

# descriptor_extractor.detect_and_extract(img2)
# keypoints2 = descriptor_extractor.keypoints
# descriptors2 = descriptor_extractor.descriptors

# # descriptor_extractor.detect_and_extract(img3)
# # keypoints3 = descriptor_extractor.keypoints
# # descriptors3 = descriptor_extractor.descriptors

# matches12 = match_descriptors(descriptors1, descriptors2, max_ratio=0.6,
#                               cross_check=True)
# # matches13 = match_descriptors(descriptors1, descriptors3, max_ratio=0.6,
#                             #   cross_check=True)
# fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(11, 8))

# plt.gray()

# plot_matches(ax[0, 0], img1, img2, keypoints1, keypoints2, matches12)
# ax[0, 0].axis('off')
# ax[0, 0].set_title("Original Image vs. Flipped Image\n"
#                    "(all keypoints and matches)")

# # plot_matches(ax[1, 0], img1, img3, keypoints1, keypoints3, matches13)
# # ax[1, 0].axis('off')
# # ax[1, 0].set_title("Original Image vs. Transformed Image\n"
# #                    "(all keypoints and matches)")

# plot_matches(ax[0, 1], img1, img2, keypoints1, keypoints2, matches12[::15],
#              only_matches=True)
# ax[0, 1].axis('off')
# ax[0, 1].set_title("Original Image vs. Flipped Image\n"
#                    "(subset of matches for visibility)")

# # plot_matches(ax[1, 1], img1, img3, keypoints1, keypoints3, matches13[::15],
# #              only_matches=True)
# ax[1, 1].axis('off')
# ax[1, 1].set_title("Original Image vs. Transformed Image\n"
#                    "(subset of matches for visibility)")

# plt.tight_layout()
# plt.show()

## Scikit-image Sift

In [7]:
import matplotlib.pyplot as plt

from skimage import transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

ImportError: cannot import name 'ransac' from 'skimage.feature' (/home/sean/mambaforge/envs/monai/lib/python3.9/site-packages/skimage/feature/__init__.py)

In [ ]:
img1, img2, img3 = test_pics
descriptor_extractor = SIFT()

descriptor_extractor.detect_and_extract(img1)
keypoints1 = descriptor_extractor.keypoints
descriptors1 = descriptor_extractor.descriptors

descriptor_extractor.detect_and_extract(img2)
keypoints2 = descriptor_extractor.keypoints
descriptors2 = descriptor_extractor.descriptors

# descriptor_extractor.detect_and_extract(img3)
# keypoints3 = descriptor_extractor.keypoints
# descriptors3 = descriptor_extractor.descriptors

matches12 = match_descriptors(descriptors1, descriptors2, max_ratio=0.6,
                              cross_check=True)
# matches13 = match_descriptors(descriptors1, descriptors3, max_ratio=0.6,
                            #   cross_check=True)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(11, 8))

plt.gray()

plot_matches(ax[0, 0], img1, img2, keypoints1, keypoints2, matches12)
ax[0, 0].axis('off')
ax[0, 0].set_title("Original Image vs. Flipped Image\n"
                   "(all keypoints and matches)")

# plot_matches(ax[1, 0], img1, img3, keypoints1, keypoints3, matches13)
# ax[1, 0].axis('off')
# ax[1, 0].set_title("Original Image vs. Transformed Image\n"
#                    "(all keypoints and matches)")

plot_matches(ax[0, 1], img1, img2, keypoints1, keypoints2, matches12[::15],
             only_matches=True)
ax[0, 1].axis('off')
ax[0, 1].set_title("Original Image vs. Flipped Image\n"
                   "(subset of matches for visibility)")

# plot_matches(ax[1, 1], img1, img3, keypoints1, keypoints3, matches13[::15],
#              only_matches=True)
ax[1, 1].axis('off')
ax[1, 1].set_title("Original Image vs. Transformed Image\n"
                   "(subset of matches for visibility)")

plt.tight_layout()
plt.show()

## Scikit-image - Ransac

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from skimage import data
from skimage.util import img_as_float
from skimage.feature import (corner_harris, corner_subpix, corner_peaks,
                             plot_matches)
from skimage.transform import warp, AffineTransform
from skimage.exposure import rescale_intensity
from skimage.measure import ransac

In [ ]:
coords_orig = corner_peaks(corner_harris(img_orig_gray), threshold_rel=0.001,
                           min_distance=5)
coords_warped = corner_peaks(corner_harris(img_warped_gray),
                             threshold_rel=0.001, min_distance=5)

# determine sub-pixel corner position
coords_orig_subpix = corner_subpix(img_orig_gray, coords_orig, window_size=9)
coords_warped_subpix = corner_subpix(img_warped_gray, coords_warped,
                                     window_size=9)


def gaussian_weights(window_ext, sigma=1):
    y, x = np.mgrid[-window_ext:window_ext+1, -window_ext:window_ext+1]
    g = np.zeros(y.shape, dtype=np.double)
    g[:] = np.exp(-0.5 * (x**2 / sigma**2 + y**2 / sigma**2))
    g /= 2 * np.pi * sigma * sigma
    return g


def match_corner(coord, window_ext=5):
    r, c = np.round(coord).astype(np.intp)
    window_orig = img_orig[r-window_ext:r+window_ext+1,
                           c-window_ext:c+window_ext+1, :]

    # weight pixels depending on distance to center pixel
    weights = gaussian_weights(window_ext, 3)
    weights = np.dstack((weights, weights, weights))

    # compute sum of squared differences to all corners in warped image
    SSDs = []
    for cr, cc in coords_warped:
        window_warped = img_warped[cr-window_ext:cr+window_ext+1,
                                   cc-window_ext:cc+window_ext+1, :]
        SSD = np.sum(weights * (window_orig - window_warped)**2)
        SSDs.append(SSD)

    # use corner with minimum SSD as correspondence
    min_idx = np.argmin(SSDs)
    return coords_warped_subpix[min_idx]


# find correspondences using simple weighted sum of squared differences
src = []
dst = []
for coord in coords_orig_subpix:
    src.append(coord)
    dst.append(match_corner(coord))
src = np.array(src)
dst = np.array(dst)


# estimate affine transform model using all coordinates
model = AffineTransform()
model.estimate(src, dst)

# robustly estimate affine transform model with RANSAC
model_robust, inliers = ransac((src, dst), AffineTransform, min_samples=3,
                               residual_threshold=2, max_trials=100)
outliers = inliers == False


# compare "true" and estimated transform parameters
print("Ground truth:")
print(f'Scale: ({tform.scale[1]:.4f}, {tform.scale[0]:.4f}), '
      f'Translation: ({tform.translation[1]:.4f}, '
      f'{tform.translation[0]:.4f}), '
      f'Rotation: {-tform.rotation:.4f}')
print("Affine transform:")
print(f'Scale: ({model.scale[0]:.4f}, {model.scale[1]:.4f}), '
      f'Translation: ({model.translation[0]:.4f}, '
      f'{model.translation[1]:.4f}), '
      f'Rotation: {model.rotation:.4f}')
print("RANSAC:")
print(f'Scale: ({model_robust.scale[0]:.4f}, {model_robust.scale[1]:.4f}), '
      f'Translation: ({model_robust.translation[0]:.4f}, '
      f'{model_robust.translation[1]:.4f}), '
      f'Rotation: {model_robust.rotation:.4f}')

# visualize correspondence
fig, ax = plt.subplots(nrows=2, ncols=1)

plt.gray()

inlier_idxs = np.nonzero(inliers)[0]
plot_matches(ax[0], img_orig_gray, img_warped_gray, src, dst,
             np.column_stack((inlier_idxs, inlier_idxs)), matches_color='b')
ax[0].axis('off')
ax[0].set_title('Correct correspondences')

outlier_idxs = np.nonzero(outliers)[0]
plot_matches(ax[1], img_orig_gray, img_warped_gray, src, dst,
             np.column_stack((outlier_idxs, outlier_idxs)), matches_color='r')
ax[1].axis('off')
ax[1].set_title('Faulty correspondences')

plt.show()